In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import re

In [2]:
#Importing the Dataset
train=pd.read_csv("C:\\Users\\Kunwar\\Desktop\\Fake News Detector\\Dataset\\CSV Dataset\\Train_Set.csv",encoding="latin1")
test=pd.read_csv("C:\\Users\\Kunwar\\Desktop\\Fake News Detector\\Dataset\\CSV Dataset\\Test_Set.csv",encoding="latin1")

In [3]:
print(train.shape)

(43837, 5)


In [4]:
#Combining Important Features
train['total']=train['Headlines']+' '+train['Summary']+' '+train['Fake/Real']

In [5]:
#Setence Tokenizing Library
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kunwar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
#Stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
#Creating Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [8]:
#Applying Lemmatizer to Training Dataset
lemmatizer=WordNetLemmatizer()
for index,row in train.iterrows():
    filter_sentence = ''
    sentence = row['total']
    
    sentence = re.sub(r'[^\w\s]','',sentence) #Removing Punctutations
    
    words = nltk.word_tokenize(sentence) #tokenization
    
    words = [w for w in words if not w in stop_words]  #Removing Stpowirds
    
    for word in words:
        filter_sentence = filter_sentence + ' ' + str(lemmatizer.lemmatize(word)).lower()
        
    train.loc[index,'total'] = filter_sentence


In [9]:
#Encoding Categorical Data (Sources)
train['Source_ID']=train['Sources'].factorize()[0]
sour_id_df = train[["Sources", "Source_ID"]].drop_duplicates().sort_values('Source_ID')
sour_to_id = dict(sour_id_df.values)
id_to_sour = dict(sour_id_df[['Source_ID','Sources']].values)

In [10]:
train = train[['total','Sources','Source_ID']]

In [11]:
train.head()

,total,Sources,Source_ID
0,bungalow private island sale 46 crore a bunga...,Bloomberg,0
1,americans react bollywood song pinga a video ...,YouTube,1
2,world toilet day celebrated november 19 the u...,United Nations,2
3,over 157 lakh people affected assam flood ove...,Northeast Today,3
4,varun gandhi unhurt stage collapse bjp mp var...,India Today,4


In [12]:
#TF-idf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
def vectorize_text(features, max_features):
    vectorizer = TfidfVectorizer( stop_words='english',
                            decode_error='strict',
                            analyzer='word',
                            ngram_range=(1, 2),
                            max_features=max_features
                            #max_df=0.5 # Verwendet im ML-Kurs unter Preprocessing                   
                            )
    feature_vec = vectorizer.fit_transform(features)
    return feature_vec.toarray()

In [20]:
#Splitting the Dataset into Train-Set and Test-Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train['total'], train['Sources'], random_state= 0)

In [14]:
#Count Vectorizer/Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [15]:
#Creating Tfidf Vectorizer from Train Set
from sklearn.feature_extraction.text import TfidfTransformer
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


In [16]:
#Fitting into Clssification Model(Linear SVC)
from sklearn.svm import LinearSVC
clf = LinearSVC().fit(X_train_tfidf, y_train)

#Predictions
from sklearn import metrics
y_pred = clf.predict(count_vect.transform(X_test))
print(metrics.accuracy_score(y_test, y_pred))

#Examining the Predictions
print(metrics.classification_report(y_test,y_pred, labels= train.Sources, target_names=train['Sources'].unique()))

0.3460766423357664


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1537: UserWarning: labels size, 43837, does not match size of target_names, 921
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                                      precision    recall  f1-score   support

                           Bloomberg       0.16      0.09      0.12        88
                             YouTube       0.57      0.57      0.57       580
                      United Nations       1.00      0.33      0.50         3
                     Northeast Today       0.00      0.00      0.00        11
                         India Today       0.12      0.13      0.12       535
                                 PTI       0.18      0.22      0.20       391
                         India Times       0.50      0.51      0.51      2092
                        The Guardian       0.28      0.21      0.24       415
                         VentureBeat       0.00      0.00      0.00        31
                  Rate My Professors       0.00      0.00      0.00         0
                  The Times of India       0.49      0.48      0.49      2054
                     Hindustan Times       0.49      0.48      

In [19]:
#Predicting Random Results
print(clf.predict(count_vect.transform(['Large Business Bankruptcy may lead to economic crisis around the world'])))

['Reuters']
